In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [3]:
# Generate SMIRNOFF ligand mol object
mol = oechem.OEMol()
molpdb = '../OpenMMCubes/tests/input/toluene.pdb'
ffxml = open('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml', 'r')
with oechem.oemolistream(molpdb) as ifs:
    oechem.OEReadMolecule(ifs, mol)
    
    # Generate molecule System, encode to bytes to attach generic data.
    from smarty.forcefield import ForceField
    mol_ff = ForceField(ffxml)
    mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
    print("System is: ",type(mol_sys))
    encoded_sys = openmm.XmlSerializer.serialize(mol_sys).encode()
    print("Encoded system is:: ",type(encoded_sys))
    
    # Generate parmed Structure
    molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys, xyz=mol_pos)
    molecule_structure.residues[0].name = "MOL"
    
    import base64, pickle
    mol_dict = molecule_structure.__getstate__()
    pkl_dict = pickle.dumps(mol_dict)
    encoded_string = base64.b64encode(pkl_dict)

    
    with oechem.oemolostream('smirff_mol.oeb.gz') as ofs:
        mol.AddData(oechem.OEGetTag('system'), encoded_sys)
        mol.AddData(oechem.OEGetTag('structure'), encoded_string)
        oechem.OEWriteMolecule(ofs, mol)
        
    ffxml.close()

System is:  <class 'simtk.openmm.openmm.System'>
Encoded system is::  <class 'bytes'>


In [60]:
datamol = oechem.OEMol()
with oechem.oemolistream('smirff_mol.oeb.gz') as ifs:
    oechem.OEReadMolecule(ifs, datamol)
    print(datamol.GetData().keys())
    
    if 'structure' in datamol.GetData().keys():
        encoded_str = datamol.GetData(oechem.OEGetTag('structure'))
        decoded_str = base64.b64decode(encoded_str)
        struct_dict = pickle.loads(decoded_str)
        
        newstruct = parmed.structure.Structure()
        newstruct.__setstate__(struct_dict)


dict_keys(['structure', 'system'])


In [61]:

print(len(struct_dict.keys()))
print(len(mol_dict.keys()))

40
40


In [59]:
import numpy as np
set(struct_dict.keys()) == set(mol_dict.keys())
for k,v in mol_dict.items():
    try:
        if type(v) == type(np.zeros([1,1])):
            print("ARRAY FOUND IN KEY:", k)
            if np.array_equal(v, struct_dict[k]):
                print(k, 'True')
        elif v == struct_dict[k]:
            print(k, 'True')
    except:
        print("EXCEPTION IN KEY:", k, type(v))
    

rb_torsion_types True
pi_torsion_types True
unknown_functional True
urey_bradleys True
groups True
cmap_types True
angles True
angle_types True
acceptors True
trigonal_angles True
out_of_plane_bend_types True
rb_torsions True
exclusions True
pi_torsions True
dihedral_types True
donors True
stretch_bends True
cmaps True
urey_bradley_types True
torsion_torsion_types True
improper_types True
adjusts True
impropers True
title True
out_of_plane_bends True
stretch_bend_types True
bond_types True
_combining_rule True
multipole_frames True
ARRAY FOUND IN KEY: _coordinates
_coordinates True
torsion_torsions True
nrexcl True
dihedrals True
adjust_types True
space_group True
chiral_frames True
_box True
trigonal_angle_types True
bonds True
